# Creating CMIP6 experiment definitions with pyesdoc

## Setup

In [1]:
# Import standard python modules
import os
import tempfile
import uuid

In [2]:
# Import pyesdoc & cim
import pyesdoc
import pyesdoc.ontologies.cim as cim

ES-DOC :: pyesdoc :: INFO > Loading pyesdoc config from: /Users/macg/dev/esdoc/ops/config/pyesdoc.conf


In [3]:
# Set default document project code.
_PROJECT = 'CMIP6-TEST'

# Set default document source.
_SOURCE = 'test-script'

# Set default I/O directory.
_IO_DIR = tempfile.mkdtemp()

## Document Creation

In [4]:
# Create author.
author = pyesdoc.create(cim.v2.Party, source=_SOURCE)
author.email = "momipsl@ipsl.jussieu.fr"
author.name = "Mark A. Greenslade"

In [5]:
# Create project.
project = pyesdoc.create(cim.v2.Project, author=author, source=_SOURCE)
project.canonical_name = _PROJECT
project.name = _PROJECT
project.long_name = "Climate model Inter-Comparason Project 6"

In [6]:
# Create experimental requirement.
tc = pyesdoc.create(cim.v2.TemporalConstraint, project=_PROJECT, author=author, source=_SOURCE)
tc.canonical_name = "stc.027"
tc.conformance_is_requested = False
tc.description = "Begin in 1979 and run for 30 years"
tc.duration = cim.v2.TimePeriod()
tc.duration.length = 30
tc.duration.units = "years"
tc.duration.date_type = "unused"
tc.name = "3.3.stc.1979_30yr"

In [8]:
# Create experiment.
experiment = pyesdoc.create(cim.v2.NumericalExperiment, project=_PROJECT, author=author, source=_SOURCE)
experiment.canonical_name = "amip"
experiment.description = "AMIP (1979 - at least 2008). Impose SSTs and sea ice from observations but with other conditions as in experiment 3.2 historical."
experiment.long_name = "AMIP"
experiment.name = "amip"
experiment.rationale = "AMIP. Baseline simulation for model evaluation."
experiment.requirements.append(tc)

## Document references

In [9]:
# CIM v1 reference
tc_reference_1 = cim.v1.DocReference()
tc_reference_1.name = 'stc.027'
tc_reference_1.id = '1e19715f-d743-4f4e-b8e9-2b1765f638e8'
tc_reference_1.version = '1'
tc_reference_1.url = 'http://archive.es-doc.org/1e19715f-d743-4f4e-b8e9-2b1765f638e8/1'

In [10]:
# CIM v2 reference
tc_reference = cim.v2.CimLink()

## Document Validation

In [12]:
# Validate document & print errors
errors = pyesdoc.validate(experiment)
for err in errors:
    print(err)

doc.requirements[0].canonical_name --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].description --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].duration.date_type --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].duration.units --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].meta.metadata_author.email --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].meta.metadata_author.name --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.requirements[0].name --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.canonical_name --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.description --> is of invalid type (actual = <type 'str'>, expected=<type 'unicode'>)
doc.long_name --> is of invalid type (ac

## Document Serialization

In [13]:
# Encode as JSON
experiment_as_json = pyesdoc.encode(experiment, pyesdoc.ESDOC_ENCODING_JSON)

In [14]:
# Encode as XML
experiment_as_xml = pyesdoc.encode(experiment, pyesdoc.ESDOC_ENCODING_XML)

In [15]:
# Encode as HTML
experiment_as_html = pyesdoc.encode(experiment, pyesdoc.ESDOC_ENCODING_HTML)

In [16]:
# Decode from JSON
experiment_from_json = pyesdoc.decode(experiment_as_json, pyesdoc.ESDOC_ENCODING_JSON)

In [17]:
# Decode from XML
experiment_from_xml = pyesdoc.decode(experiment_as_xml, pyesdoc.ESDOC_ENCODING_XML)

## Document I/O

In [18]:
# Set I/O directory.
pyesdoc.set_option("output_dir", _IO_DIR)

In [19]:
# Write to file system.
experiment_fpath_json = pyesdoc.write(experiment, pyesdoc.ESDOC_ENCODING_JSON)
experiment_fpath_xml = pyesdoc.write(experiment, pyesdoc.ESDOC_ENCODING_XML)

In [20]:
# Read from file system.
experiment_from_json_file = pyesdoc.read(experiment_fpath_json)
experiment_from_xml_file = pyesdoc.read(experiment_fpath_xml)

## Document Publishing

In [21]:
# Publish to ES-DOC web-service
pyesdoc.publish(experiment)

ValueError: Cannot publish invalid documents.

In [ ]:
# Retrieve from ES-DOC web-service
# ... retrieve latest version
experiment = pyesdoc.retrieve(experiment.meta.id)
# ... retrieve specific version
experiment = pyesdoc.retrieve(experiment.meta.id, experiment.meta.version)

In [ ]:
# Republish to ES-DOC web-service
# ... update document
experiment.name += "-UPDATED"
# ... cache current document version
experiment_version = experiment.meta.version
# ... republish
pyesdoc.publish(experiment)
# ... assert document version was updated
assert experiment_version + 1 == experiment.meta.version

In [ ]:
# Unpublish.
pyesdoc.unpublish(experiment.meta.id)

# Assert document can no longer be retrieved from web-service.
assert pyesdoc.retrieve(experiment.meta.id) is None